In [9]:
import numpy as np
import pandas as pd

In [10]:
df = pd.read_csv('AirPass.zip')

In [11]:
# ЗАДАНИЕ 6.1

df.isnull().sum().sum()

310

In [12]:
# ЗАДАНИЕ 6.2

df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median())
df['Arrival Delay in Minutes'].mean()

15.133392362180475

In [13]:
# ЗАДАНИЕ 6.3

print(df.groupby('Gender')['satisfaction'].value_counts()*100/df.shape[0])

print(df.groupby('Type of Travel')['satisfaction'].value_counts()*100/df.shape[0])

print(df.groupby('Class')['satisfaction'].value_counts()*100/df.shape[0])

Gender  satisfaction           
Female  neutral or dissatisfied    29.058554
        satisfied                  21.687327
Male    neutral or dissatisfied    27.608177
        satisfied                  21.645942
Name: satisfaction, dtype: float64
Type of Travel   satisfaction           
Business travel  satisfied                  40.177472
                 neutral or dissatisfied    28.785225
Personal Travel  neutral or dissatisfied    27.881506
                 satisfied                   3.155798
Name: satisfaction, dtype: float64
Class     satisfaction           
Business  satisfied                  33.184478
          neutral or dissatisfied    14.614452
Eco       neutral or dissatisfied    36.614567
          satisfied                   8.374076
Eco Plus  neutral or dissatisfied     5.437712
          satisfied                   1.774715
Name: satisfaction, dtype: float64


In [14]:
# чтобы сработало 
 
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

In [15]:
# ЗАДАНИЕ 6.4

df=pd.get_dummies(df)
df.shape

(103904, 27)

In [20]:
# хз откуда взялась

df = df.drop('Unnamed: 0', axis=1)

In [22]:
# ЗАДАНИЕ 6.5
from sklearn.model_selection import train_test_split

X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=26)
y_test.shape

(20781,)

In [23]:
# ЗАДАНИЕ 6.6

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

0.9408251379303

In [25]:
# ЗАДАНИЕ 6.7

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
preds_test = model_lr.predict(X_test)
f1_score(preds_test, y_test)

0.8546883773161146

In [26]:
# ЗАДАНИЕ 6.8

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

model_ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=26),random_state=26,learning_rate=0.01)

model_ada.fit(X_train, y_train)
preds_test = model_ada.predict(X_test)
f1_score(preds_test, y_test)

0.9404794558121674

In [27]:
# ЗАДАНИЕ 6.9

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

model_for_gs = GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring=make_scorer(f1_score),
                  verbose=5)
 
gs.fit(X_train, y_train)
 
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.873 total time=   0.1s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.870 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.871 total time=   0.1s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.880 total time=   0.3s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.878 total time=   0.2s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.875 total time=   0.2s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.901 total time=   0.5s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.896 total time=   0.6s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.897 total time=   0.5s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.2s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.920 total time=   1.2s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

In [28]:
# ЗАДАНИЕ 6.10

from xgboost import XGBClassifier

model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train,y_train)
preds_test = model_xgb.predict(X_test)
f1_score(preds_test, y_test)

0.9579785161685312

In [ ]:
# ЗАДАНИЕ 6.11

from catboost import CatBoostClassifier, Pool

model = CatBoostClassifier(random_state=26)
model.fit(X_train, y_train)
preds_class = model.predict(X_test)
f1_score(preds_class, y_test)

# 0.9598221728257851

In [ ]:
# ЗАДАНИЕ 6.12

from catboost.utils import get_confusion_matrix

cm = get_confusion_matrix(model, Pool(X_train, y_train))

# array([[46668.,   538.],
#       [ 1255., 34662.]])

In [ ]:
# ЗАДАНИЕ 6.13

pd.DataFrame(
    {
        "feature_importance": clf.get_feature_importance(),
        "feature_names": df.drop(columns="satisfaction").columns,
    }
).sort_values(by=["feature_importance"], ascending=False)

#

***

In [46]:
df = pd.read_csv('Frogs_MFCCs.zip')
X = df.drop(['Family', 'Genus', 'Species', 'RecordID'], axis=1)
y = df['Family'].apply(lambda x: 1 if x=='Dendrobatidae' else 0)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=31)
y_test.shape

(1439,)

In [48]:
# ЗАДАНИЕ 7.5

from sklearn.ensemble import RandomForestClassifier

model_rfc = RandomForestClassifier(10, random_state=42)
model_rfc.fit(X_train, y_train)
pred = model_rfc.predict(X_test)
f1_score(y_test, pred)

0.9726775956284154

In [49]:
def is_numpy_c_ordered(X):
    return hasattr(X, "flags") and X.flags.c_contiguous

In [50]:
# ЗАДАНИЕ 7.6

from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=31)),
              ('knn', KNeighborsClassifier(n_neighbors= 11)),
              ('nb', GaussianNB() )               
]
metamodel = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
metamodel.fit(X_train, y_train)
pred = metamodel.predict(X_test)
f1_score(y_test, pred)

0.989247311827957